In [1]:
import os
from imageio import imread
from pathlib import Path
import numpy as np

In [2]:
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

(train_images_org, train_labels), (test_images_org, test_labels) = datasets.cifar10.load_data()
# train_images_img, test_images_img = train_images_org.astype(dtype=np.complex64) / 255.0, test_images_org.astype(dtype=np.complex64) / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

2023-08-05 15:33:28.471192: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 15:33:29.439223: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
class dataset:
    def __init__:
        

In [3]:
from tensorflow.keras import models
import tensorflow.keras.layers as layers

def get_tf_sequential_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(32, 32, 3)))   # Always use ComplexInput at the start
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(10, activation='softmax'))
    return model

get_tf_sequential_model().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

2023-08-05 15:33:35.048807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-05 15:33:35.072549: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-05 15:33:35.072744: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
import cvnn.layers as complex_layers

def get_cvnn_sequential_model():
    model = models.Sequential()
    model.add(complex_layers.ComplexInput(input_shape=(32, 32, 3)))   # Always use ComplexInput at the start
    model.add(complex_layers.ComplexConv2D(32, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    model.add(complex_layers.ComplexConv2D(64, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    model.add(complex_layers.ComplexConv2D(64, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexFlatten())
    model.add(complex_layers.ComplexDense(64, activation="cart_relu"))
    model.add(complex_layers.ComplexDense(10, activation='softmax_real_with_abs'))
    return model

get_cvnn_sequential_model().summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 complex_conv2d (ComplexCon  (None, 30, 30, 32)        1792      
 v2D)                                                            
                                                                 
 complex_max_pooling2d (Com  (None, 15, 15, 32)        0         
 plexMaxPooling2D)                                               
                                                                 
 complex_conv2d_1 (ComplexC  (None, 13, 13, 64)        36992     
 onv2D)                                                          
                                                                 
 complex_max_pooling2d_1 (C  (None, 6, 6, 64)          0         
 omplexMaxPooling2D)                                             
                                                                 
 complex_conv2d_2 (ComplexC  (None, 4, 4, 64)         

#### Adding a imaginary component into the real data and running the value and using CVNN

In [5]:
# #Running the CVNN model on the data

# from tensorflow import losses, metrics

# model_cvnn = get_cvnn_sequential_model()
# model_cvnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    
# history = model_cvnn.fit(train_images_img, train_labels, epochs=100, validation_data=(test_images_img, test_labels),batch_size=32)

2023-08-05 15:33:41.736274: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1228800000 exceeds 10% of free system memory.
2023-08-05 15:33:42.830852: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1228800000 exceeds 10% of free system memory.


Epoch 1/100


2023-08-05 15:33:44.937838: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-05 15:33:45.347641: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fea227ece30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-05 15:33:45.347674: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-08-05 15:33:45.411980: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 [==============================] - 31s 18ms/step - loss: 1.9396 - accuracy: 0.2953 - val_loss: 1.8023 - val_accuracy: 0.3456
Epoch 2/100
1563/1563 [==============================] - 26s 17ms/step - loss: 1.5331 - accuracy: 0.4456 - val_loss: 1.4249 - val_accuracy: 0.4902
Epoch 3/100
1563/1563 [==============================] - 27s 17ms/step - loss: 1.3594 - accuracy: 0.5148 - val_loss: 1.4158 - val_accuracy: 0.4895
Epoch 4/100
1563/1563 [==============================] - 28s 18ms/step - loss: 1.2435 - accuracy: 0.5590 - val_loss: 1.2773 - val_accuracy: 0.5366
Epoch 5/100
1563/1563 [==============================] - 27s 17ms/step - loss: 1.1524 - accuracy: 0.5931 - val_loss: 1.2116 - val_accuracy: 0.5692
Epoch 6/100
1563/1563 [==============================] - 29s 19ms/step - loss: 1.0746 - accuracy: 0.6232 - val_loss: 1.0971 - val_accuracy: 0.6109
Epoch 7/100
1563/1563 [==============================] - 26s 17ms/step - loss: 1.0083 - accuracy: 0.6470 - val_loss: 1.0941 - val_

KeyboardInterrupt: 

In [ ]:
#Running an ANN on the data while ignoring the imaginary values

model_ann = get_tf_sequential_model()
model_ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    
history = model_ann.fit(train_images_img, train_labels, epochs=100, validation_data=(test_images_img, test_labels),batch_size=32)

#### Now mapping the data from the real plane to the complex plane using Hilberts transform

In [ ]:
import scipy

train_images_hil, test_images_hil = scipy.signal.hilbert(train_images_org / 255), scipy.signal.hilbert(test_images_org / 255)

In [ ]:
#Running the CVNN model on the hilbert data

from tensorflow import losses, metrics

model_cvnn = get_cvnn_sequential_model()
model_cvnn.compile(optimizer='sgd', loss='categorical_crossentropy', metrics='accuracy')
    
history = model_cvnn.fit(train_images_hil, train_labels, epochs=100, validation_data=(test_images_hil, test_labels),batch_size=32)

1563/1563 [==============================] - ETA: 0s - loss: 1.9206 - accuracy: 0.3009

2023-07-12 23:32:18.703953: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.
2023-07-12 23:32:29.283290: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 468.75MiB (rounded to 491520000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-07-12 23:32:29.283414: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-07-12 23:32:29.283454: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 57, Chunks in use: 57. 14.2KiB allocated for chunks. 14.2KiB in use in bin. 3.8KiB client-requested in use in bin.
2023-07-12 23:32:29.283486: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B a

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.